# Referenced in the DB_CORRECT_AND_TRANSFORM BOOK

# After intial exploration of the DB out of this it looks good - reasonable number of games and values

In [1]:
# Dependencies
import os
import sys
import pandas as pd
import sqlite3

import sqlite3
##################### DESTRUCTIVE - DO NOT RUN WITHOUT BACKUP OF DB FILE #####################
# Replace 'your_database.db' with the path to your database file


db_path = '../data/db/Combined_DB_v2.db'

def remove_duplicates(conn, table_name):
    """
    Remove duplicate rows from a specified table.
    """
    cursor = conn.cursor()
    
    # Fetching column names for the duplication check
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = [info[1] for info in cursor.fetchall()]
    
    # Properly quote column names to handle special characters or reserved keywords
    columns_quoted = [f'"{col}"' for col in columns]
    columns_joined = ', '.join(columns_quoted)
    
    query = f"""
    DELETE FROM {table_name}
    WHERE rowid NOT IN (
        SELECT MIN(rowid)
        FROM {table_name}
        GROUP BY {columns_joined}
    )
    """
    try:
        cursor.execute(query)
        conn.commit()
        print(f"Duplicates removed from {table_name}.")
    except sqlite3.OperationalError as e:
        print(f"Failed to remove duplicates from {table_name}: {e}")

def main(db_path):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Fetch all table names in the database
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    
    for table_name in tables:
        remove_duplicates(conn, table_name[0])
    
    print("Done. All duplicates have been removed.")
    conn.close()

# Call the main function
if __name__ == "__main__":
    main(db_path)


Duplicates removed from scoring_summary.
Duplicates removed from penalty_summary.
Duplicates removed from goalie_stats.
Duplicates removed from player_stats.
Duplicates removed from linescore.
Duplicates removed from game_details.
Duplicates removed from master_roster.
Duplicates removed from player_stats_ytd.
Duplicates removed from line_chart.
Duplicates removed from advanced_metrics.
Done. All duplicates have been removed.
